# About This Notebook

This notebook was done on Kaggle and downloaded after. It contains code that we use to scrape the games data from the 1st to 10k rows of our list of games. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/steam60kgamesdata/gamesdata.csv


In [2]:
df = pd.read_csv('/kaggle/input/steam60kgamesdata/gamesdata.csv')

In [3]:
from bs4 import BeautifulSoup
import time
import requests
import json
from tqdm import tqdm

In [4]:
df.head()

Name  \
0            Persona 3 Portable   
1                         Boing   
2                    Fraymakers   
3  【VR】Physical Exam / イタズラ身体測定   
4           FPS Seduce - Hentai   

                                                Link  Release Date  \
0  https://store.steampowered.com/app/1809700/Per...  18 Jan, 2023   
1  https://store.steampowered.com/app/2180390/Boi...  18 Jan, 2023   
2  https://store.steampowered.com/app/1420350/Fra...  18 Jan, 2023   
3  https://store.steampowered.com/app/2262110/VRP...  18 Jan, 2023   
4  https://store.steampowered.com/app/1853780/FPS...  18 Jan, 2023   

                            Price  
0                      CDN$ 27.99  
1   -20%\n\n\nCDN$ 10.49CDN$ 8.39  
2                      CDN$ 22.79  
3  -10%\n\n\nCDN$ 25.99CDN$ 23.39  
4    -10%\n\n\nCDN$ 5.69CDN$ 5.12

In [5]:
counter = 0
data = []

for counter in tqdm(range(0,10000)):
        hlink = df['Link'][counter]
        response = requests.get(hlink)
        soup1 = BeautifulSoup(response.text, 'html.parser')

        description = soup1.find('div',id='game_area_description')
        description = description.text.strip().replace('\r','').replace('\t','').replace('\n','') if description else None

        reviews = soup1.find('div', class_='user_reviews_summary_row')
        reviews = reviews.text.strip().replace('\r','').replace('\t','').replace('\n','') if reviews else None
        tags_div = soup1.find('div', class_='glance_tags popular_tags')
        tags = tags_div.find_all('a', class_='app_tag') if tags_div else 'N/A'

        if tags != 'N/A':
            tagged = []
            for tag in tags:
                tagged.append(tag.text.strip())
        else:
            tagged = 'N/A'

        tagged_string = ', '.join(tagged)
        data.append([description, reviews, tagged_string])
        
        counter=+1
    

100%|██████████| 10000/10000 [1:34:25<00:00,  1.77it/s]


In [6]:
data_df = pd.DataFrame(data, columns=['description','reviews','tags'])
data_df

description  \
0     About This GameExperience the iconic, critical...   
1     About This GameLong ago in the deepest and dar...   
2     About This GameWhat is Fraymakers?Fraymakers i...   
3     About This GameThis game is for Meta Quest2 on...   
4     About This GameMake love to many beautiful ani...   
...                                                 ...   
9995  About This GameWhen we talk about outer space ...   
9996  About This GameFrom Cloisters Interactive, A M...   
9997  About This GameWorld of Contraptions is a phys...   
9998  About This GameWelcome to Teraphobia: The Begi...   
9999  About This GameThe Lost and The Wicked is a Br...   

                                                reviews  \
0     All Reviews:Mostly Positive(771)- 74% of the 7...   
1     All Reviews:1 user reviews- Need more user rev...   
2     All Reviews:Mixed(214)- 68% of the 214 user re...   
3     All Reviews:1 user reviews- Need more user rev...   
4     All Reviews:2 user reviews- Need more user rev...   
...                                                 ...   
9995  All Reviews:5 user reviews- Need more user rev...   
9996  Recent Reviews:Very Positive(17)- 88% of the 1...   
9997  All Reviews:Very Positive(73)- 90% of the 73 u...   
9998  All Reviews:5 user reviews- Need more user rev...   
9999  All Reviews:Positive(15)- 86% of the 15 user r...   

                                                   tags  
0     RPG, Story Rich, Anime, Turn-Based Combat, JRP...  
1     Casual, Action, Arcade, Atmospheric, 2D, Singl...  
2     Indie, Fighting, 2D Fighter, Multiplayer, Acti...  
3     Simulation, VR, Dating Sim, Cute, Immersive Si...  
4     Nudity, Sexual Content, Hentai, Romance, Anime...  
...                                                 ...  
9995  Adventure, Point & Click, Space, 2D, Clicker, ...  
9996  Adventure, Casual, Interactive Fiction, 2D, 3D...  
9997  Simulation, Indie, Strategy, Building, Sandbox...  
9998  Action, Singleplayer, Multiplayer, Horror, Psy...  
9999  Twin Stick Shooter, Action-Adventure, Top-Down...  

[10000 rows x 3 columns]

In [7]:
data_df.to_csv('first_ten_k_games.csv', index=False)